In [2]:
import os
import cv2
import numpy as np
from tqdm import tqdm

import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from torch.utils.data import Dataset, DataLoader, WeightedRandomSampler
from torch import amp

import timm
from sklearn.metrics import roc_auc_score, average_precision_score, f1_score
import pandas as pd
import matplotlib.pyplot as plt

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print("Using device:", device)

torch.backends.cudnn.benchmark = True


Using device: cuda


In [3]:
# 1. Load the NIH ChestX-ray14 metadata
df = pd.read_csv("Data_Entry_2017.csv")

# 2. Define all 14 disease classes
all_labels = [
    'Atelectasis','Cardiomegaly','Effusion','Infiltration','Mass',
    'Nodule','Pneumonia','Pneumothorax','Consolidation','Edema',
    'Emphysema','Fibrosis','Pleural_Thickening','Hernia'
]

# 3. Encode labels
def encode_labels(label_str):
    vec = [0] * 14
    if label_str != "No Finding":
        for disease in label_str.split('|'):
            if disease in all_labels:
                vec[all_labels.index(disease)] = 1
    return vec

df['labels'] = df['Finding Labels'].apply(encode_labels)
labels_expanded = pd.DataFrame(df['labels'].tolist(), columns=all_labels)
df_clean = pd.concat([df[['Image Index']], labels_expanded], axis=1)

# Save clean CSV
df_clean.to_csv("labels_clean.csv", index=False)
print("✅ Saved labels_clean.csv with shape:", df_clean.shape)



✅ Saved labels_clean.csv with shape: (112120, 15)


In [4]:
import albumentations as A
from albumentations.pytorch import ToTensorV2

# Train augmentations
train_transform = A.Compose([
    A.HorizontalFlip(p=0.5),
    A.Rotate(limit=10, p=0.5),
    A.RandomBrightnessContrast(p=0.5),
    A.CLAHE(p=0.3),
    A.CoarseDropout(max_holes=1, hole_height=32, hole_width=32, p=0.3),
    A.Normalize(mean=(0.485, 0.456, 0.406),
                std=(0.229, 0.224, 0.225)),
    ToTensorV2(),
])

# Validation transforms
val_transform = A.Compose([
    A.Normalize(mean=(0.485, 0.456, 0.406),
                std=(0.229, 0.224, 0.225)),
    ToTensorV2(),
])


/tmp/ipykernel_1603/2590350891.py:10: UserWarning: Argument(s) 'max_holes, hole_height, hole_width' are not valid for transform CoarseDropout
  A.CoarseDropout(max_holes=1, hole_height=32, hole_width=32, p=0.3),


In [5]:
class ChestXrayDataset(Dataset):
    def __init__(self, file_list, img_dir, labels_csv, transform=None):
        self.img_paths = []
        self.labels = []

        df = pd.read_csv(labels_csv)
        label_dict = dict(zip(df['Image Index'], df.drop(columns=['Image Index']).values.tolist()))

        with open(file_list, "r") as f:
            for line in f:
                fname = line.strip().split()[0]
                if fname not in label_dict:
                    raise ValueError(f"No labels for {fname}")

                found_path = None
                for subdir in os.listdir(img_dir):
                    subpath = os.path.join(img_dir, subdir, "images", fname)
                    if os.path.exists(subpath):
                        found_path = subpath
                        break

                if found_path is None:
                    raise FileNotFoundError(f"Image {fname} not found in {img_dir}")

                self.img_paths.append(found_path)
                self.labels.append(label_dict[fname])

        self.labels = np.array(self.labels, dtype=np.float32)
        self.transform = transform

    def __len__(self):
        return len(self.img_paths)

    def __getitem__(self, idx):
        img = cv2.imread(self.img_paths[idx], cv2.IMREAD_GRAYSCALE)
        if img is None:
            raise FileNotFoundError(f"Failed to load {self.img_paths[idx]}")

        img = cv2.resize(img, (CONFIG["img_size"], CONFIG["img_size"]))
        img = np.stack([img, img, img], axis=-1)

        if self.transform:
            img = self.transform(image=img)["image"]
        else:
            img = torch.tensor(img/255.0, dtype=torch.float32).permute(2,0,1)

        label = torch.tensor(self.labels[idx], dtype=torch.float32)
        return img, label


In [6]:
CONFIG = {
    "img_size": 380,        # EfficientNet-B4 input size
    "batch_size": 8,        # reduce if OOM
    "num_workers": 6,
    "epochs": 50,
    "lr": 1e-4,
    "checkpoint_dir": "checkpoints_efficientnet_b4",  # separate dir for B4
}
os.makedirs(CONFIG["checkpoint_dir"], exist_ok=True)

# File names for best models (avoid conflicts with DenseNet/B3)
BEST_MODEL_AUROC = os.path.join(CONFIG["checkpoint_dir"], "best_model_auroc_b4.pth")
BEST_MODEL_F1    = os.path.join(CONFIG["checkpoint_dir"], "best_model_f1_b4.pth")


In [7]:
labels_csv = "labels_clean.csv"

train_dataset = ChestXrayDataset("train_split.txt", "data", labels_csv, transform=train_transform)
val_dataset   = ChestXrayDataset("val_split.txt", "data", labels_csv, transform=val_transform)

# Weighted Sampler
labels = train_dataset.labels
class_counts = labels.sum(axis=0)
class_weights = 1.0 / (class_counts + 1e-6)
sample_weights = (labels * class_weights).sum(axis=1)
sample_weights = np.clip(sample_weights, 1e-6, None)

sampler = WeightedRandomSampler(
    weights=torch.DoubleTensor(sample_weights),
    num_samples=len(sample_weights),
    replacement=True
)

train_loader = DataLoader(train_dataset, batch_size=CONFIG["batch_size"], sampler=sampler,
                          num_workers=CONFIG["num_workers"], pin_memory=True, persistent_workers=True)
val_loader = DataLoader(val_dataset, batch_size=CONFIG["batch_size"], shuffle=False,
                        num_workers=CONFIG["num_workers"], pin_memory=True, persistent_workers=True)

print(f"Train size: {len(train_dataset)}, Val size: {len(val_dataset)}")

# pos_weight for BCE loss
pos = labels.sum(axis=0)
neg = len(labels) - pos
pos_weight = torch.tensor(neg / (pos + 1e-6), dtype=torch.float32).to(device)


Train size: 77871, Val size: 8653


In [8]:
model = timm.create_model("efficientnet_b4", pretrained=True, num_classes=14)
model = model.to(device).to(memory_format=torch.channels_last)

# Loss
criterion = nn.BCEWithLogitsLoss(pos_weight=pos_weight)

# Optimizer
optimizer = optim.AdamW(model.parameters(), lr=CONFIG["lr"], weight_decay=1e-4)

# Scheduler with Warmup
from transformers import get_cosine_schedule_with_warmup
total_steps = len(train_loader) * CONFIG["epochs"]
warmup_steps = int(0.1 * total_steps)

scheduler = get_cosine_schedule_with_warmup(
    optimizer,
    num_warmup_steps=warmup_steps,
    num_training_steps=total_steps
)

# AMP scaler
scaler = amp.GradScaler("cuda")


In [11]:
def compute_metrics(y_true, y_probs, threshold=0.5, class_names=None):
    C = y_true.shape[1]
    aucs, aps, f1s = [], [], []
    per_class_metrics = {}

    for c in range(C):
        try:
            auc = roc_auc_score(y_true[:, c], y_probs[:, c])
        except:
            auc = np.nan
        try:
            ap = average_precision_score(y_true[:, c], y_probs[:, c])
        except:
            ap = np.nan

        y_pred_bin = (y_probs[:, c] >= threshold).astype(int)
        f1 = f1_score(y_true[:, c], y_pred_bin, zero_division=0)

        aucs.append(auc)
        aps.append(ap)
        f1s.append(f1)

        if class_names:
            per_class_metrics[class_names[c]] = {
                "auroc": auc,
                "auprc": ap,
                "f1": f1
            }
        else:
            per_class_metrics[c] = {"auroc": auc, "auprc": ap, "f1": f1}

    macro_f1 = np.nanmean(f1s)
    micro_f1 = f1_score(y_true, (y_probs >= threshold).astype(int), average='micro', zero_division=0)

    return {
        "auroc_macro": np.nanmean(aucs),
        "ap_macro": np.nanmean(aps),
        "macro_f1": macro_f1,
        "micro_f1": micro_f1,
        "per_class": per_class_metrics
    }


# ========================= Training Loop =========================
best_val_auroc = -1
best_val_f1 = -1
patience = 10
epochs_no_improve = 0

class_names = [
    'Atelectasis','Cardiomegaly','Effusion','Infiltration','Mass',
    'Nodule','Pneumonia','Pneumothorax','Consolidation','Edema',
    'Emphysema','Fibrosis','Pleural_Thickening','Hernia'
]

for epoch in range(CONFIG["epochs"]):
    print(f"\nEpoch {epoch+1}/{CONFIG['epochs']}")
    print(f"Current LR: {scheduler.get_last_lr()[0]:.6f}")

    # ---- Training ----
    model.train()
    running_loss = 0.0
    for images, labels in tqdm(train_loader, desc="Training", leave=False):
        images, labels = images.to(device, non_blocking=True), labels.to(device, non_blocking=True)

        optimizer.zero_grad()
        with amp.autocast("cuda"):
            outputs = model(images)
            loss = criterion(outputs, labels)

        scaler.scale(loss).backward()
        scaler.step(optimizer)
        scaler.update()

        scheduler.step()  # ✅ step per batch

        running_loss += loss.item()

    avg_train_loss = running_loss / len(train_loader)

    # ---- Validation ----
    model.eval()
    val_loss, all_probs, all_targets = 0.0, [], []
    with torch.no_grad():
        for images, labels in tqdm(val_loader, desc="Validation", leave=False):
            images, labels = images.to(device, non_blocking=True), labels.to(device, non_blocking=True)
            with amp.autocast("cuda"):
                outputs = model(images)
                loss = criterion(outputs, labels)

            val_loss += loss.item()
            all_probs.append(torch.sigmoid(outputs).cpu().numpy())
            all_targets.append(labels.cpu().numpy())

    avg_val_loss = val_loss / len(val_loader)
    all_probs = np.vstack(all_probs)
    all_targets = np.vstack(all_targets)

    metrics = compute_metrics(all_targets, all_probs, class_names=class_names)

    print(f"Train Loss: {avg_train_loss:.4f} | Val Loss: {avg_val_loss:.4f} | "
          f"AUROC: {metrics['auroc_macro']:.3f} | AUPRC: {metrics['ap_macro']:.3f} | "
          f"Macro-F1: {metrics['macro_f1']:.3f} | Micro-F1: {metrics['micro_f1']:.3f}")

    print("Sample per-class AUROC:")
    for disease in ["Hernia", "Fibrosis", "Edema"]:
        val = metrics["per_class"][disease]["auroc"]
        print(f"  {disease}: {val:.3f}")

    # ---- Save checkpoint for every epoch ----
    torch.save(model.state_dict(), os.path.join(CONFIG["checkpoint_dir"], f"epoch_{epoch+1}.pth"))

    improved = False

    # ✅ Best by AUROC
    if metrics["auroc_macro"] > best_val_auroc:
        best_val_auroc = metrics["auroc_macro"]
        torch.save(model.state_dict(), BEST_MODEL_AUROC)
        print(f"🌟 Best model updated (AUROC {best_val_auroc:.3f})")
        improved = True

    # ✅ Best by Macro-F1
    if metrics["macro_f1"] > best_val_f1:
        best_val_f1 = metrics["macro_f1"]
        torch.save(model.state_dict(), BEST_MODEL_F1)
        print(f"🌟 Best model updated (Macro-F1 {best_val_f1:.3f})")
        improved = True

    # ---- Early stopping ----
    if improved:
        epochs_no_improve = 0
    else:
        epochs_no_improve += 1
        print(f"⚠️ No improvement for {epochs_no_improve} epoch(s).")

    if epochs_no_improve >= patience:
        print(f"⏹ Early stopping triggered after {epoch+1} epochs "
              f"(no AUROC/F1 improvement for {patience} epochs).")
        break



Epoch 1/50
Current LR: 0.000000


Training:   0%|          | 0/9734 [00:00<?, ?it/s]/home/prem/PROJECT DATA/.venv/lib/python3.12/site-packages/torch/optim/lr_scheduler.py:224: UserWarning: Detected call of `lr_scheduler.step()` before `optimizer.step()`. In PyTorch 1.1.0 and later, you should call them in the opposite order: `optimizer.step()` before `lr_scheduler.step()`.  Failure to do this will result in PyTorch skipping the first value of the learning rate schedule. See more details at https://pytorch.org/docs/stable/optim.html#how-to-adjust-learning-rate
  warnings.warn(
Training:  45%|████▍     | 4349/9734 [08:22<08:48, 10.19it/s] libpng warning: iCCP: profile 'ICC Profile': 'GRAY': Gray color space not permitted on RGB PNG


Train Loss: 2.7577 | Val Loss: 1.6851 | AUROC: 0.689 | AUPRC: 0.097 | Macro-F1: 0.090 | Micro-F1: 0.092
Sample per-class AUROC:
  Hernia: 0.705
  Fibrosis: 0.661
  Edema: 0.844
🌟 Best model updated (AUROC 0.689)
🌟 Best model updated (Macro-F1 0.090)

Epoch 2/50
Current LR: 0.000020


Training:  40%|████      | 3902/9734 [06:37<09:49,  9.88it/s]libpng warning: iCCP: profile 'ICC Profile': 'GRAY': Gray color space not permitted on RGB PNG


Train Loss: 1.8845 | Val Loss: 1.4816 | AUROC: 0.747 | AUPRC: 0.148 | Macro-F1: 0.104 | Micro-F1: 0.112
Sample per-class AUROC:
  Hernia: 0.819
  Fibrosis: 0.725
  Edema: 0.871
🌟 Best model updated (AUROC 0.747)
🌟 Best model updated (Macro-F1 0.104)

Epoch 3/50
Current LR: 0.000040


...

Train Loss: 0.0204 | Val Loss: 8.3849 | AUROC: 0.726 | AUPRC: 0.180 | Macro-F1: 0.256 | Micro-F1: 0.303
Sample per-class AUROC:
  Hernia: 0.597
  Fibrosis: 0.698
  Edema: 0.771
⚠️ No improvement for 1 epoch(s).

Epoch 47/50
Current LR: 0.000002


Training:  82%|████████▏ | 7991/9734 [13:22<02:57,  9.80it/s]libpng warning: iCCP: profile 'ICC Profile': 'GRAY': Gray color space not permitted on RGB PNG


Train Loss: 0.0213 | Val Loss: 8.6329 | AUROC: 0.724 | AUPRC: 0.179 | Macro-F1: 0.256 | Micro-F1: 0.302
Sample per-class AUROC:
  Hernia: 0.602
  Fibrosis: 0.696
  Edema: 0.770
⚠️ No improvement for 2 epoch(s).

Epoch 48/50
Current LR: 0.000001


Training:  73%|███████▎  | 7059/9734 [11:48<04:30,  9.89it/s]libpng warning: iCCP: profile 'ICC Profile': 'GRAY': Gray color space not permitted on RGB PNG


KeyboardInterrupt: 

In [10]:
class_names = [
    'Atelectasis','Cardiomegaly','Effusion','Infiltration','Mass',
    'Nodule','Pneumonia','Pneumothorax','Consolidation','Edema',
    'Emphysema','Fibrosis','Pleural_Thickening','Hernia'
]

BEST_MODEL_AUROC = "checkpoints_efficientnet_b4/best_model_auroc_b4.pth"
BEST_MODEL_F1    = "checkpoints_efficientnet_b4/best_model_f1_b4.pth"

def evaluate_model(model_path, model_name):
    """
    Load a model, run inference, tune thresholds, and compute metrics.
    Returns dict with overall + per-class metrics.
    """
    # Load model
    model = timm.create_model("efficientnet_b4", pretrained=False, num_classes=14)
    model.load_state_dict(torch.load(model_path, map_location=device))
    model = model.to(device)
    model.eval()

    all_probs, all_targets = [], []
    with torch.no_grad():
        for images, labels in tqdm(val_loader, desc=f"Eval {model_name}", leave=False):
            images = images.to(device, non_blocking=True)
            with amp.autocast("cuda"):
                outputs = model(images)
            all_probs.append(torch.sigmoid(outputs).cpu().numpy())
            all_targets.append(labels.numpy())

    all_probs = np.vstack(all_probs)
    all_targets = np.vstack(all_targets)

    # ---- Per-class threshold tuning ----
    best_thresh = []
    per_class_results = {}
    for c, disease in enumerate(class_names):
        best_f, best_t = -1, 0.5
        for t in np.linspace(0.05, 0.95, 19):
            f = f1_score(all_targets[:,c], (all_probs[:,c] >= t).astype(int), zero_division=0)
            if f > best_f:
                best_f, best_t = f, t
        best_thresh.append(best_t)

        # compute per-class metrics with tuned threshold
        y_true_c = all_targets[:,c]
        y_prob_c = all_probs[:,c]
        y_pred_c = (y_prob_c >= best_t).astype(int)

        try:
            auc = roc_auc_score(y_true_c, y_prob_c)
        except:
            auc = np.nan
        try:
            ap = average_precision_score(y_true_c, y_prob_c)
        except:
            ap = np.nan
        f1c = f1_score(y_true_c, y_pred_c, zero_division=0)

        per_class_results[disease] = {
            "best_thresh": best_t,
            "auroc": auc,
            "auprc": ap,
            "f1": f1c
        }

    # ---- Overall metrics ----
    final_preds = (all_probs >= np.array(best_thresh)).astype(int)
    macro_f1 = f1_score(all_targets, final_preds, average='macro', zero_division=0)
    micro_f1 = f1_score(all_targets, final_preds, average='micro', zero_division=0)
    auroc_macro = roc_auc_score(all_targets, all_probs, average='macro')

    results = {
        "model": model_name,
        "macro_auroc": auroc_macro,
        "macro_f1": macro_f1,
        "micro_f1": micro_f1,
        "per_class": per_class_results,
    }
    return results


# ========================= Evaluate both best models =========================
results_auroc = evaluate_model(BEST_MODEL_AUROC, "EfficientNet-B4 Best AUROC")
results_f1    = evaluate_model(BEST_MODEL_F1, "EfficientNet-B4 Best F1")

# ---- Print Summary ----
def print_summary(results):
    print(f"\n📊 {results['model']}")
    print(f"Macro AUROC: {results['macro_auroc']:.3f}")
    print(f"Macro-F1 (tuned): {results['macro_f1']:.3f}")
    print(f"Micro-F1 (tuned): {results['micro_f1']:.3f}")
    print("\nPer-class metrics:")
    for disease, metrics in results["per_class"].items():
        print(f"  {disease:<20} "
              f"Thresh={metrics['best_thresh']:.2f} | "
              f"AUROC={metrics['auroc']:.3f} | "
              f"AUPRC={metrics['auprc']:.3f} | "
              f"F1={metrics['f1']:.3f}")

print_summary(results_auroc)
print_summary(results_f1)

# ---- Comparison table ----
import pandas as pd

df_auroc = pd.DataFrame(results_auroc["per_class"]).T
df_f1    = pd.DataFrame(results_f1["per_class"]).T

comparison_df = df_auroc[["auroc","auprc","f1"]].rename(
    columns={"auroc":"AUROC (best AUROC model)",
             "auprc":"AUPRC (best AUROC model)",
             "f1":"F1 (best AUROC model)"})

comparison_df["AUROC (best F1 model)"] = df_f1["auroc"]
comparison_df["AUPRC (best F1 model)"] = df_f1["auprc"]
comparison_df["F1 (best F1 model)"] = df_f1["f1"]

print("\n================ Performance Comparison per class ================\n")
display(comparison_df.round(3))


/tmp/ipykernel_1603/3056760698.py:17: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  model.load_state_dict(torch.load(model_path, map_location=device))
/tmp/ipykernel_1603/30


📊 EfficientNet-B4 Best AUROC
Macro AUROC: 0.763
Macro-F1 (tuned): 0.230
Micro-F1 (tuned): 0.267

Per-class metrics:
  Atelectasis          Thresh=0.85 | AUROC=0.747 | AUPRC=0.256 | F1=0.331
  Cardiomegaly         Thresh=0.95 | AUROC=0.861 | AUPRC=0.183 | F1=0.217
  Effusion             Thresh=0.85 | AUROC=0.860 | AUPRC=0.462 | F1=0.486
  Infiltration         Thresh=0.60 | AUROC=0.616 | AUPRC=0.243 | F1=0.300
  Mass                 Thresh=0.95 | AUROC=0.760 | AUPRC=0.192 | F1=0.259
  Nodule               Thresh=0.90 | AUROC=0.646 | AUPRC=0.136 | F1=0.197
  Pneumonia            Thresh=0.95 | AUROC=0.603 | AUPRC=0.019 | F1=0.044
  Pneumothorax         Thresh=0.95 | AUROC=0.858 | AUPRC=0.233 | F1=0.302
  Consolidation        Thresh=0.95 | AUROC=0.677 | AUPRC=0.090 | F1=0.138
  Edema                Thresh=0.95 | AUROC=0.873 | AUPRC=0.118 | F1=0.183
  Emphysema            Thresh=0.95 | AUROC=0.899 | AUPRC=0.276 | F1=0.256
  Fibrosis             Thresh=0.95 | AUROC=0.738 | AUPRC=0.081 | F1=0

,AUROC (best AUROC model),AUPRC (best AUROC model),F1 (best AUROC model),AUROC (best F1 model),AUPRC (best F1 model),F1 (best F1 model)
Atelectasis,0.747,0.256,0.331,0.738,0.262,0.341
Cardiomegaly,0.861,0.183,0.217,0.838,0.180,0.303
Effusion,0.860,0.462,0.486,0.842,0.401,0.472
Infiltration,0.616,0.243,0.300,0.585,0.210,0.288
Mass,0.760,0.192,0.259,0.758,0.239,0.340
Nodule,0.646,0.136,0.197,0.672,0.155,0.235
Pneumonia,0.603,0.019,0.044,0.598,0.021,0.058
Pneumothorax,0.858,0.233,0.302,0.867,0.309,0.398
Consolidation,0.677,0.090,0.138,0.651,0.093,0.170
Edema,0.873,0.118,0.183,0.782,0.107,0.199
